<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%EC%9D%B4%ED%83%88%EC%98%88%EC%B8%A1(%EB%B6%84%EB%A5%98)_4%EB%AA%A8%EB%8D%B8(%EC%9D%98%EC%82%AC%EA%B2%B0%EC%A0%95%2C%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%2C%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8%2C%EC%84%9C%ED%8F%AC%ED%8A%B8%EB%B2%A1%ED%84%B0%EB%A8%B8%EC%8B%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.datamanim.com/dataset/03_dataq/typetwo.html
#분류
#서비스 이탈예측 데이터
#데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
x_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv", stringsAsFactors = T)
y_train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv", stringsAsFactors = T)
x_test <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv", stringsAsFactors = T)
x_label <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv", stringsAsFactors = T)
library(dplyr)
library(caret)
library(car)
library(lmtest)
library(ROCR)
#install.packages("ModelMetrics")
#install.packages("randomForest")
#install.packages("rpart")
library(ModelMetrics)
library(randomForest)
library(rpart)
library(e1071)
# 결측치 확인
#apply(is.na(x_train),2,sum)
#apply(is.na(x_test),2,sum)
#apply(is.na(y_train),2,sum)

x_train<-merge(x_train,y_train,by='CustomerId')
x_test<-merge(x_test,x_label,by='CustomerId')
x_test<-x_test[,c(3,6,7,8,9,10,11,12,13)]
x_train<-x_train[,c(3,6,7,8,9,10,11,12,13)]


#x_test$Exited<-as.factor(x_test$Exited)
#x_train$Exited<-as.factor(x_train$Exited)

m1 <- rpart(Exited~., data=x_train) #의사결정나무
m2 <- glm(Exited~., data=x_train, family="binomial") #로지스틱 회귀분석
m3 <- randomForest(Exited~., data=x_train, ntree=100) #랜덤포레스트
m4 <- svm(Exited~., data=x_train)#서포트벡터머신

pd1<-predict(m1,newdata=x_test)
pd2<-predict(m2,newdata=x_test)
pd3<-predict(m3,newdata=x_test, type = "response")
pd4<-predict(m4,newdata=x_test)

prd1 <- prediction(pd1, x_test$Exited)
prd2 <- prediction(pd2, x_test$Exited)
prd3 <- prediction(pd3, x_test$Exited)
prd4 <- prediction(pd4, x_test$Exited)
pf1<-performance(prd1, measure = "auc")
pf2<-performance(prd2, measure = "auc")
pf3<-performance(prd3, measure = "auc")
pf4<-performance(prd4, measure = "auc")
cat("ctree auc:",pf1@y.values[[1]],'\n')
cat("glm auc:",pf2@y.values[[1]],'\n')
cat("randomForest auc:",pf3@y.values[[1]],'\n')
cat("svm auc:",pf4@y.values[[1]],'\n')

ERROR: ignored